# Inapproximability

[Arxiv Paper](https://arxiv.org/pdf/2205.15874.pdf)

In [ ]:
import numpy as np
import utils

## Theorem 5.6 ($\ell \le 0$)

Goal: 
$$
\min_{0\le \kappa\le 1, \ell_q\le 0, \ell_p\le 0}[\max_{0\le q\le 1, 0\le p}[(1-\kappa)2q(1-q)+\kappa 2(1-q)(1-e^{-p})+2p\ell_p+2q\ell_q]-\beta (\ell_p+\ell_q)].
$$
Consider
$$
(1-\kappa)2q(1-q)+\kappa 2(1-q)(1-e^{-p})+2p\ell_p+2q\ell_q.
$$
Derivative wrt $p$: 
$$
\kappa 2(1-q)e^{-p}+2\ell_p
$$
Derivative wrt $q$:
$$
2(1-\kappa)(1-2q)-\kappa 2(1-e^{-p})+2\ell_q
$$

In [ ]:
def maximize_unbounded_brute_force(kappa, ell_p, ell_q, verbose=False):
    """Brute forces over a range of q to get the approximate maximum over all p and q"""
    assert ell_p < 0
    def get_val(p, q):
        return (
            (1 - kappa) * 2 * q * (1 - q)
            + 2 * kappa * (1 - q) * (1 - np.exp(-p))
            + 2 * p * ell_p
            + 2 * q * ell_q
        )

    best_p = 0
    best_q = 0
    max_val = 0
    for q in np.linspace(0, 1, 1001):  # TODO: replace this?
        if np.isclose(q, 1) or np.isclose(kappa, 0):
            p = 0
        else:
            p = max(-np.log(-ell_p / kappa / (1 - q)), 0)
        val = get_val(p, q)
        if val > max_val:
            max_val = val
            best_p = p
            best_q = q
    if verbose:
        print("best_p =", best_p, "best_q =", best_q)
    return max_val

from numpy.polynomial import Polynomial

def maximize_unbounded(kappa, ell_p, ell_q, verbose =False):
    """Returns the max over all p and q"""
    assert ell_p < 0
    ans = 0

    def get_val(p, q):
        return (
            (1 - kappa) * 2 * q * (1 - q)
            + 2 * kappa * (1 - q) * (1 - np.exp(-p))
            + 2 * p * ell_p
            + 2 * q * ell_q
        )

    def consider(p, q):
        nonlocal ans
        if not ((0 <= p) and (0 <= q <= 1)):
            return
        if verbose:
            print("consider", p, q)
        ans = max(ans, get_val(p, q))

    def maximize_unbounded_q_0():
        # 2 * kappa * e^{-p} + 2 * ell_p
        # p = 0 or kappa * e^{-p} = -ell_p
        consider(0, 0)
        if kappa > utils.EPS:
            consider(-np.log(-ell_p / kappa), 0)


    def maximize_unbounded_q_1():
        consider(0, 1)

    def maximize_unbounded_p_0():
        """
        (1-2q) = -\ell_q / (1-\kappa)
        """
        if np.isclose(kappa, 1):
            return
        q = (1 + ell_q / (1- kappa)) / 2
        consider(0, q)

    def maximize_unbounded_remainder():
        """
        e^{-p} = -ell_p / (\kappa (1-q))
        2(1-\kappa)(1-2q)-\kappa 2(1-e^{-p})+2\ell_q = 0
        2(1-\kappa)(1-2q)-\kappa 2(1+ell_p / (\kappa (1-q)))+2\ell_q = 0
        2(1-\kappa)(1-2q)*(1-q)-\kappa 2((1-q)+ell_p / (\kappa))+2\ell_q * (1-q) = 0
        """
        if np.isclose(kappa, 0):
            return
        poly_q = 2 * (1 - kappa) * Polynomial([1, -2]) * Polynomial([1, -1])
        poly_q -= kappa * 2 * (Polynomial([1 + ell_p / kappa, -1]))
        poly_q += 2 * ell_q * Polynomial([1, -1])
        if verbose:
            print("poly", poly_q)
        for q in poly_q.roots():
            if verbose:
                print("q_cand", q)
            if np.isreal(q):
                p = -np.log(-ell_p / (kappa * (1-q)))
                consider(p, q)


    maximize_unbounded_q_0()
    maximize_unbounded_q_1()
    maximize_unbounded_p_0()
    maximize_unbounded_remainder()
    return ans
   

In [ ]:
def sanity_check(kappa, ell_p, ell_q):
    ans_0 = maximize_unbounded_brute_force(kappa, ell_p, ell_q)
    ans_1 = maximize_unbounded(kappa, ell_p, ell_q)
    dif = np.abs(ans_0 - ans_1)
    return dif <= 1e-5

In [ ]:
def maximize_unbounded_sanity_check(kappa: float):
    print(f"Checking {kappa}\n", end='')
    max_dif = 1e-6
    for ell_p in np.linspace(-1, -0.1, 10):
        print(f"Doing {kappa} {ell_p}\n", end='')
        for ell_q in np.linspace(-1, 1, 21):
            if not sanity_check(kappa, ell_p, ell_q):
                print(f"Failed {kappa}\n", end='')
                return
    print(f"Done {kappa}\n", end='')

In [ ]:
import multiprocess

# with multiprocess.Pool(processes=2) as pool:
#     pool.map(maximize_unbounded_sanity_check, np.linspace(0, 1, 11))

In [ ]:
def get_val_neg_ell(beta, kappa, ell_p, ell_q):
    if kappa < 0 or kappa > 1 or ell_p > 0 or ell_q > 0:
        return 1
    return maximize_unbounded(kappa, ell_p, ell_q) - beta * (ell_p + ell_q)


def get_val_unconstrained_ell(beta, kappa, ell_p, ell_q):
    if kappa < 0 or kappa > 1 or ell_p > 0:
        return 1
    return maximize_unbounded(kappa, ell_p, ell_q) - beta * (ell_p + ell_q)

In [ ]:
maximize_unbounded_brute_force(0.3, -0.1, 0)

0.3784823341902765

In [ ]:
maximize_unbounded(0.3, -0.1, 0)

0.37848233641417156

In [ ]:
maximize_unbounded_brute_force(0.3515, -0.1294, 0)

0.3479022780815806

In [ ]:
maximize_unbounded(0.3515, -0.1294, 0)

0.3479025004647751

## Section 5 Table 2

In [ ]:
def hill_climb(f, args, verbose = False):
    """naive algorithm for finding a local minimum of f given a starting point args"""
    inc = 1
    for i in range(20):
        if verbose:
            print("Climbing", i)
        flag = True
        while flag:
            flag = False
            if verbose:
                print("Considering", args, f(*args))
            for j in range(len(args)):
                new_args = args.copy()
                new_args[j] += inc
                new_val = f(*new_args)
                if new_val < f(*args):
                    flag = True
                    args = new_args
                    
                new_args = args.copy()
                new_args[j] -= inc
                new_val = f(*new_args)
                if new_val < f(*args):
                    flag = True
                    args = new_args
        inc /= 2
    return args

In [ ]:
table = """
0.1 & 0.2750 & 0.0935 & 0.6705 & -0.6095 & -0.2680 \\
0.2 & 0.2998 & 0.1743 & 0.6513 & -0.5322 & -0.2192 \\
0.3 & 0.3245 & 0.2433 & 0.6498 & -0.4705 & -0.1505 \\
0.4 & 0.3488 & 0.3008 & 0.6506 & -0.4207 & -0.0893 \\
0.5 & 0.3728 & 0.3477 & 0.6484 & -0.3800 & -0.0410 \\
0.6 & 0.3964 & 0.3846 & 0.6388 & -0.3400 & 0.0000 \\
0.7 & 0.4195 & 0.4162 & 0.5811 & -0.2887 & 0.0000 \\
0.8 & 0.4420 & 0.4420 & 0.5088 & -0.2289 & 0.0000 \\
0.9 & 0.4621 & 0.4621 & 0.4335 & -0.1766 & 0.0000 \\
1.0 & 0.4773 & 0.4773 & 0.3515 & -0.1294 & 0.0000 \\
"""

for line in table.strip().split("\\")[:-1]:
    beta, _, alpha, kappa, ell_p, ell_q = list(map(lambda x: float(x.strip()), line.strip().split('&')))
    kappa, ell_p, ell_q = hill_climb(lambda kappa, ell_p, ell_q: get_val_neg_ell(beta, kappa, ell_p, ell_q), [kappa, ell_p, ell_q])
    num = 4
    kappa = round(kappa, num)
    ell_p = round(ell_p, num)
    ell_q = round(ell_q, num)
    new_line = [beta, _, round(get_val_neg_ell(beta, kappa, ell_p, ell_q), 4), kappa, ell_p, ell_q]
    for i in range(1, len(new_line)):
        new_line[i] = str(f"{new_line[i]:.4f}")
    new_line[0] = str(new_line[0])
    print(' & '.join(new_line)+" \\\\")

0.1 & 0.2750 & 0.0935 & 0.6705 & -0.6095 & -0.2680 \\
0.2 & 0.2998 & 0.1743 & 0.6514 & -0.5323 & -0.2191 \\
0.3 & 0.3245 & 0.2433 & 0.6497 & -0.4705 & -0.1506 \\
0.4 & 0.3488 & 0.3008 & 0.6505 & -0.4207 & -0.0894 \\
0.5 & 0.3728 & 0.3477 & 0.6484 & -0.3797 & -0.0407 \\
0.6 & 0.3964 & 0.3846 & 0.6388 & -0.3400 & 0.0000 \\
0.7 & 0.4195 & 0.4162 & 0.5809 & -0.2885 & 0.0000 \\
0.8 & 0.4420 & 0.4420 & 0.5082 & -0.2283 & 0.0000 \\
0.9 & 0.4621 & 0.4621 & 0.4317 & -0.1755 & 0.0000 \\
1.0 & 0.4773 & 0.4773 & 0.3513 & -0.1292 & 0.0000 \\


## Theorem 8.4 (Conference Theorem 7), Section 8 Table 4 (Conference Table 1)

In [ ]:
def hillclimb2(beta, kappa, ell_p, ell_q):
    f = lambda kappa, ell_p, ell_q: get_val_unconstrained_ell(beta, kappa, ell_p, ell_q)
    # print("Starting", f(kappa, ell_p, ell_q))
    # assert False
    step = 0.0001
    while True:
        # print("Starting", f(kappa, ell_p, ell_q), kappa, ell_p, ell_q)
        min_val = 1.0
        best_config = None
        for nkappa in np.linspace(kappa - 3 * step, kappa + 3 * step, 7):
            for nell_p in np.linspace(ell_p - 3 * step, ell_p + 3 * step, 7):
                for nell_q in np.linspace(ell_q - 3 * step, ell_q + 3 * step, 7):
                    val = f(nkappa, nell_p, nell_q)
                    if val < min_val:
                        min_val = val
                        best_config = (nkappa, nell_p, nell_q)
        if (kappa, ell_p, ell_q) == best_config:
            step /= 2
            if step < 0.00001:
                break
        # ori_val = f(kappa, ell_p, ell_q)
        # print("ori_val", ori_val)
        kappa, ell_p, ell_q = best_config
        # assert f(kappa, ell_p, ell_q) < ori_val

    return min_val, best_config

def gen_table_4():
    table = """
0.1 & 0.0935 & 0.6708 & -0.6064 & -0.2644 \\
0.2 & 0.1743 & 0.6598 & -0.5370 & -0.2102 \\
0.3 & 0.2432 & 0.6577 & -0.4809 & -0.1504 \\
0.4 & 0.3008 & 0.6519 & -0.4261 & -0.0940 \\
0.5 & 0.3476 & 0.6465 & -0.3778 & -0.0407 \\
0.6 & 0.3844 & 0.6390 & -0.3309 & 0.0128 \\
0.7 & 0.4114 & 0.6315 & -0.2881 & 0.0655 \\
0.8 & 0.4293 & 0.6226 & -0.2498 & 0.1155 \\
0.9 & 0.4383 & 0.6144 & -0.2142 & 0.1668 \\
1.0 & 0.4390 & 0.6022 & -0.1819 & 0.2152 \\
"""

    for line in table.strip().split("\\")[:-1]:
        beta, alpha, kappa, ell_p, ell_q = list(map(lambda x: float(x.strip()), line.strip().split('&')))
        # print("OOPS", get_val_unconstrained_ell(beta, kappa, ell_p, ell_q))
        # kappa, ell_p, ell_q = hill_climb(lambda kappa, ell_p, ell_q: get_val_unconstrained_ell(beta, kappa, ell_p, ell_q), [kappa, ell_p, ell_q])
        kappa, ell_p, ell_q = hillclimb2(beta, kappa, ell_p, ell_q)[1]
        num = 4
        kappa = round(kappa, num)
        ell_p = round(ell_p, num)
        ell_q = round(ell_q, num)
        print("value =", get_val_unconstrained_ell(beta, kappa, ell_p, ell_q))
        new_line = [beta, round(get_val_unconstrained_ell(beta, kappa, ell_p, ell_q), num), kappa, ell_p, ell_q]
        for i in range(1, len(new_line)):
            new_line[i] = str(f"{new_line[i]:.4f}")
        new_line[0] = str(new_line[0])
        print(' & '.join(new_line)+" \\\\")

gen_table_4()
        

value = 0.09346645863384453
0.1 & 0.0935 & 0.6707 & -0.6066 & -0.2647 \\
value = 0.1742922172799287
0.2 & 0.1743 & 0.6602 & -0.5372 & -0.2097 \\
value = 0.24318754821778454
0.3 & 0.2432 & 0.6574 & -0.4806 & -0.1505 \\
value = 0.30077477940912517
0.4 & 0.3008 & 0.6518 & -0.4260 & -0.0940 \\
value = 0.3476455954982463
0.5 & 0.3476 & 0.6464 & -0.3778 & -0.0407 \\
value = 0.384350062300983
0.6 & 0.3844 & 0.6389 & -0.3308 & 0.0128 \\
value = 0.4113985104998959
0.7 & 0.4114 & 0.6309 & -0.2879 & 0.0652 \\
value = 0.4292512269679634
0.8 & 0.4293 & 0.6231 & -0.2499 & 0.1157 \\
value = 0.4383361869846536
0.9 & 0.4383 & 0.6144 & -0.2142 & 0.1668 \\
value = 0.43902225907462333
1.0 & 0.4390 & 0.6026 & -0.1820 & 0.2152 \\


## Theorem 8.5 (Conference Theorem 8)

In [ ]:
# we want the minimum k such that the following expression is <= 0 for all k
# this shows (2k+eps, 1)-inapproximability

def f(p):
    return (1-exp(-p))**2 - k*2*p

# we can find k using wolfram alpha: https://www.wolframalpha.com/input?i=maximum+%5Cfrac%7B%5Cleft%281-e%5E%7B-x%7D%5Cright%29%5E%7B2%7D%7D%7B2x%7D
# k ~ 0.20363

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6cbc651c-00ab-4e5c-98d9-adde0f700e67' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>